## Chapter 3,4 5

In [5]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-3.0.0-preview2-bin-hadoop2.7'

In [7]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [12]:
spark.conf.set("spark.sql.shuffle.partitions","5")

In [13]:
staticDataFrame = spark.read.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("C:/Users/pilla/Documents/Spark-The-Definitive-Guide-master/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")

In [16]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [14]:
from pyspark.sql.functions import window, column,desc, col
staticDataFrame\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost",
"InvoiceDate")\
.groupBy(
col("CustomerId"), window(col("InvoiceDate")," 1 day"))\
.sum("total_cost")\
.show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   14075.0|[2011-12-04 19:00...|316.78000000000003|
|   18180.0|[2011-12-04 19:00...|            310.73|
|   15358.0|[2011-12-04 19:00...| 830.0600000000003|
|   15392.0|[2011-12-04 19:00...|304.40999999999997|
|   15290.0|[2011-12-04 19:00...|263.02000000000004|
+----------+--------------------+------------------+
only showing top 5 rows



#### Streaming Way

In [17]:
streamingDataFrame = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger",1)\
.format("csv")\
.load("C:/Users/pilla/Documents/Spark-The-Definitive-Guide-master/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")

In [18]:
streamingDataFrame.isStreaming

True

In [21]:
purchaseByCustomerPerHour = streamingDataFrame\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost",
"InvoiceDate")\
.groupBy(
col("CustomerId"), window(col("InvoiceDate"),"1 day"))\
.sum("total_cost")

In [22]:
purchaseByCustomerPerHour.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()

In [24]:
spark.sql("""SELECT * FROM customer_purchases
          ORDER BY `sum(total_cost)` DESC """)\
.show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   18102.0|[2010-12-06 19:00...|          25920.37|
|      null|[2010-12-09 19:00...|25399.560000000012|
|      null|[2010-12-16 19:00...|25375.189999999766|
|      null|[2010-12-05 19:00...|23395.099999999904|
|      null|[2010-12-02 19:00...| 23021.99999999999|
+----------+--------------------+------------------+
only showing top 5 rows



In [25]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



#### Machine learning with Pyspark

In [26]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
.na.fill(0)\
.withColumn("day_of_week",date_format(col("InvoiceDate"),"EEEE"))\
.coalesce(5)

In [28]:
trainDataFrame = preppedDataFrame.where("InvoiceDate <'2011-07-01'")
testDataFrame = preppedDataFrame.where("InvoiceDate >='2011-07-01'")

In [31]:
trainDataFrame.count()
testDataFrame.count()

296006

In [42]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
.setInputCol("day_of_week")\
.setOutputCol("day_of_week_indexed")

#### One Hot Encoding

In [43]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
.setInputCol("day_of_week_indexed")\
.setOutputCol("day_of_week_encoded")

In [44]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler()\
.setInputCols(["UnitPrice","Quantity","day_of_week_encoded"])\
.setOutputCol("features")

In [45]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline()\
.setStages([indexer,encoder,vectorAssembler])

In [46]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [47]:
transformedTraining = fittedPipeline.transform(trainDataFrame).cache()

In [49]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
.setK(20)\
.setSeed(10)

In [50]:
kmModel = kmeans.fit(transformedTraining)

In [51]:
transformedTest = fittedPipeline.transform(testDataFrame)

In [55]:
df = spark.range(500).toDF("number")
df.select(df["number"]+ 10).show()

+-------------+
|(number + 10)|
+-------------+
|           10|
|           11|
|           12|
|           13|
|           14|
|           15|
|           16|
|           17|
|           18|
|           19|
|           20|
|           21|
|           22|
|           23|
|           24|
|           25|
|           26|
|           27|
|           28|
|           29|
+-------------+
only showing top 20 rows



In [56]:
spark.range(2).collect()

[Row(id=0), Row(id=1)]

In [57]:
from pyspark.sql.types import *
b= ByteType()

In [66]:
spark.read.format("json").load("C:/Users/pilla/Documents/Spark-The-Definitive-Guide-master/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [67]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



#### Enforce specific schema on a DataFrame

In [69]:
from pyspark.sql.types import StructField, StructType,StringType, LongType

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME",StringType(),True),
    StructField("ORIGIN_COUNTRY_NAME",StringType(),True),
    StructField("count",LongType(),False, metadata={"hello":"world"})
])
df = spark.read.format("json").schema(myManualSchema)\
.load("C:/Users/pilla/Documents/Spark-The-Definitive-Guide-master/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")

In [76]:
from pyspark.sql.functions import col,column
col("someColumnName")
column("someColumnName")

Column<b'someColumnName'>

#### Accessing a DataFrame's Columns

In [79]:
spark.read.format("json").load("C:/Users/pilla/Documents/Spark-The-Definitive-Guide-master/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json").columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

#### Access First Row in DataFrame

In [80]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

#### Creating Rows

In [83]:
from pyspark.sql import Row
myRow = Row("Hello",None,1,False)

In [87]:
myRow[0], myRow[2]

('Hello', 1)

#### Creating DataFrame

In [89]:
df = spark.read.format("json").load("C:/Users/pilla/Documents/Spark-The-Definitive-Guide-master/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("dfTable")

### Creating DataFrame by specifying rows and columns

In [90]:
from pyspark.sql import Row
from pyspark.sql.types import StructField,StructType,StringType,LongType
myManualSchema = StructType([
    StructField("some",StringType(),True),
    StructField("col",StringType(),True),
    StructField("names",LongType(),False)
])
myRow = Row("Hello",None,1)
myDf = spark.createDataFrame([myRow],myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



#### Select Statement

In [91]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



#### Select Multiple Columns

In [96]:
df.select("DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



#### SelectExpr

In [97]:
from pyspark.sql.functions import expr,col,column
df.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [98]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [100]:
df.select(expr("DEST_COUNTRY_NAME AS destination").alias("DEST_COUNTRY_NAME")).show(2)

+----------------+
|DES_COUNTRY_NAME|
+----------------+
|   United States|
|   United States|
+----------------+
only showing top 2 rows



### SelectExpr together

In [101]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName","DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [102]:
df.selectExpr(
"*",
"(DEST_COUNTRY_NAME =ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [104]:
df.selectExpr("avg(count)","count(distinct (DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [115]:
from pyspark.sql.functions import lit
df.select(expr("*"),lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [116]:
df.selectExpr("*","1 as One").show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



#### Adding Columns

In [117]:
df.withColumn("numberOne",lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



#### ADD Boolean Column

In [119]:
df.withColumn("withinCountry",expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [121]:
df.selectExpr("*","ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME as withinCountry").show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



#### Renaming Columns

In [122]:
df.withColumnRenamed("DEST_COUNTRY_NAME","dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

In [123]:
dfWithLongColName = df.withColumn("This Long Column-Name",expr("ORIGIN_COUNTRY_NAME"))

In [126]:
dfWithLongColName.selectExpr("`This Long Column-Name`","`This Long Column-Name`as `new col`")\
.show(2)

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



In [127]:
dfWithLongColName.select(expr("`This Long Column-Name`")).columns

['This Long Column-Name']

#### Make Spark Case Sensitive 

In [131]:
spark.sql("set spark.sql.caseSensitive true")

DataFrame[key: string, value: string]

#### Remove Columns from DataFrame

In [132]:
df.drop("DEST_COUNTRY_NAME").columns

['ORIGIN_COUNTRY_NAME', 'count']

#### Drop Multiple Columns

In [133]:
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME")

DataFrame[count: bigint, This Long Column-Name: string]

#### Change a Column's Type (Cast)

In [134]:
df.withColumn("count2",col("count").cast("long"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: bigint]

#### Filter on DataFrame

In [135]:
df.filter(col("count")<2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



#### Using Where instead of filter

In [136]:
df.where(col("count")<2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



#### Multiple Filters on DataFrame (Chain them instead of using And Operator)

In [137]:
df.where(col("count")<2).where(col("ORIGIN_COUNTRY_NAME") !="Croatia")\
.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



#### Getting Unique Rows

In [138]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

In [139]:
df.select("ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME").distinct().count()

256

#### Generating Random Samples

In [141]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement,fraction,seed).count()

138

#### Random Splits on DataFrame

In [142]:
dataFrames = df.randomSplit([0.25,0.75],seed)
dataFrames[0].count() > dataFrames[1].count()

False

#### Concatenating and Appending Rows (Union)

In [145]:
from pyspark.sql import Row
schema = df.schema
newRows =[
    Row("New Country","Other Country",5),
    Row("New Country 2","Other Country",1)
]
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows,schema)

In [147]:
df.union(newDF)\
.where("count =1")\
.where(col("ORIGIN_COUNTRY_NAME") != "United States")\
.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|      Other Country|    1|
+-----------------+-------------------+-----+



#### Sorting Rows

In [148]:
df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



#### Using Order By

In [149]:
df.orderBy("count","DEST_COUNTRY_NAME").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [150]:
from pyspark.sql.functions import desc, asc
df.orderBy(expr("count desc")).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [151]:
df.orderBy(col("count").desc(),col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



#### An advanced tip is to use asc_nulls_first, desc_nulls_first, asc_nulls_last, or desc_nulls_last to specify where you would like your null values to appear in an ordered DataFrame

#### For Optimization purposes, it's sometimes advisable to sort within each partition before another set of transformations

In [154]:
spark.read.format("json").load("C:/Users/pilla/Documents/Spark-The-Definitive-Guide-master/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")\
.sortWithinPartitions("count").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
|    United States|          Gibraltar|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



#### Limit

In [156]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [157]:
df.orderBy(expr("count desc")).limit(6).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



#### Repartition

In [159]:
df.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [160]:
df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

#### Manually Specify Number of Partition

In [161]:
df.repartition(5, col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

#### Coalesce

In [162]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

#### Collecting Rows to the Driver

In [165]:
collectDF=df.limit(10)
collectDF.take(5)
collectDF.show()


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [166]:
collectDF.show(5,False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [167]:
collectDF.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]